In [2]:
from PIL import Image, ImageFilter
from google.cloud import vision
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import base64
import io
import os

#Funciones generales para uso global
import numpy as np
import time
from shapely.geometry import Polygon, Point
import geopandas as gpd
from shapely.ops import cascaded_union
import rasterio
from rasterio.mask import mask
from shapely import geometry
import pandas as pd
from geotiff import GeoTiff


#Funciones empleadas en la detección de texto
import os, sys
import numpy as np
import cv2
import time
from imutils.object_detection import non_max_suppression

import rasterio
from rasterio.enums import Resampling

# Model v1: Split the image, detect text areas, and convert to text strings

## Splitting the image

In [63]:
# Takes a Rasterio dataset and splits it into squares of dimensions squareDim * squareDim
def splitImageIntoCells(img, filename, num_imgs=3):
    squareDim_wide = img.shape[1] // num_imgs
    squareDim_height = img.shape[0] // num_imgs
    
    numberOfCellsWide = img.shape[1] // squareDim_wide
    numberOfCellsHigh = img.shape[0] // squareDim_height
    x, y = 0, 0
    count = 0
    for hc in range(numberOfCellsHigh):
        y = hc * squareDim_height
        for wc in range(numberOfCellsWide):
            x = wc * squareDim_wide
            geom = getTileGeom(img.transform, x, y, squareDim_wide, squareDim_height)
            getCellFromGeom(img, geom, filename, count)
            count = count + 1

# Generate a bounding box from the pixel-wise coordinates using the original datasets transform property
def getTileGeom(transform, x, y, squareDim_wide, squareDim_height):
    corner1 = (x, y) * transform
    corner2 = (x + squareDim_wide, y + squareDim_height) * transform
    return geometry.box(corner1[0], corner1[1],
                        corner2[0], corner2[1])

# Crop the dataset using the generated box and write it out as a GeoTIFF
def getCellFromGeom(img, geom, filename, count):
    crop, cropTransform = mask(img, [geom], crop=True)
    writeImageAsGeoTIFF(crop,
                        cropTransform,
                        img.meta,
                        img.crs,
                        filename+"_"+str(count))

# Write the passed in dataset as a GeoTIFF
def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": 'EPSG:4686'})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [6]:
def detect_text(bytes_img):
    """
    Uses the Google Vision API to extract text from
    an image.
    
    Arguments
    ---------
    file_path: str
               path of the image to process.
    
    Outputs
    -------
    response: AnnotateImageResponse object
              json like format with bounding box and other
              relevant information.
    text: str
          text extracted from the image.
    """
    client = vision.ImageAnnotatorClient()    
    image = vision.Image(content=bytes_img)
    response = client.document_text_detection(image=image)
    text = response.full_text_annotation.text
    
    return response, text


def getbytesimg_from_path(filepath):
    """
    Obtains the bytes base64 format of an image from a 
    local file path.
    
    Arguments
    ---------
    filepath: str
              Path of the image file to convert
    
    Output
    ------
    bytes_img: bytes
               base64 format of the image
    """
    with open(filepath, "rb") as image_file:
        bytes_img = image_file.read()
    
    return bytes_img

In [92]:
def split_images(filepath, num_imgs=3):
    """
    Split a large image into a grid of 3x3
    smaller images.
    
    Arguments:
    ---------
    filepath: str
              file path of the large image
    num_imgs: int (optional)
              Number of rows and columns of the grid
              
    Output
    ------
    None
    
    """
    img = rasterio.open(filepath)
    splitImageIntoCells(img, "split_out/output_data")

In [145]:
def geotif_to_jpeg(tif_filename):
    """
    Converts geotif image to jpeg.
    
    Arguments
    
    """
    with rasterio.open(tif_filename) as infile:    
        profile = infile.profile    
        profile['driver']='JPEG'
        jpeg_filename = tif_filename.replace(".tif", ".jpeg")
        with rasterio.open(jpeg_filename, 'w', **profile) as dst:
            dst.write(infile.read())

In [176]:
def get_text_coords(response):
    palabras_google = []
    boundings_google = []
    confidence_google = []
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            palabra_google = ''
            boundings_google.append(block.bounding_box)
            confidence_google.append(block.confidence)
            for parrafos in block.paragraphs:
                for palabras in parrafos.words:
                    for simbolo in palabras.symbols:
                        palabra_google = palabra_google+simbolo.text
                    palabra_google = palabra_google+' '
            palabras_google.append(palabra_google.rstrip())
            
    return palabras_google, boundings_google, confidence_google

In [168]:
def get_coords(boundings, word_index, point_index, coord):
    if coord == "x":
        return boundings[word_index].vertices[point_index].x
    else:
        return boundings[word_index].vertices[point_index].y

# Environment Variables

In [ ]:
img_to_process = "geotiffs/M-1390 F-42290.tif"

In [150]:
split_images(img_to_process)
image_to_proces = os.listdir(path='split_out')

for path in image_to_proces:
    if ('.tif' in path):
        geotif_to_jpeg(path)
        sub_image_jpeg = path.replace('.tif','.jpeg')
        response, text = detect_text(getbytesimg_from_path(sub_image_jpeg))
        words, boundings, confidence = get_text_coords(response)
        img = rasterio.open(sub_image_jpeg)
        
        
        
        #Generating the sub_geometries
        geometries = []
        centroids = []

        for i in range(len(words)):
            aux_polygon = Polygon([img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x")),
                                   img.xy(get_coords(boundings,i,1,"y"),get_coords(boundings,i,1,"x")),
                                   img.xy(get_coords(boundings,i,2,"y"),get_coords(boundings,i,2,"x")),
                                   img.xy(get_coords(boundings,i,3,"y"),get_coords(boundings,i,3,"x")),
                                   img.xy(get_coords(boundings,i,0,"y"),get_coords(boundings,i,0,"x"))])
            geometries.append(aux_polygon)
            centroids.append(aux_polygon.representative_point())

        point = gpd.GeoDataFrame(crs:str(img.crs))
        point["geometry"] = geometries
        point["toponimo_ocr"] = words
        point["confidence"] = confidence
        point["centroide_longitud"] = [x.coords[0][0] for x in centroids]
        point["centroide_latitud"] = [x.coords[0][1] for x in centroids]
        name_geometrie = 'geometries_out/'+sub_image_jpeg.replace('.jpeg','.geojson')
        try:
            point.to_file(name_geometrie, driver="GeoJSON")
        except:
            print('Empety text detect, empety geometries generated')
    else:
        continue

<ipython-input-63-2599675af965>:20: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
<ipython-input-63-2599675af965>:21: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim_wide, y + squareDim_height) * transform


## Union all subgeometries generated

In [177]:
geometries_to_proces = os.listdir(path='geometries_out')
org_img = rasterio.open(img_to_process)
rectangles = []
lista_toponimos_final = gpd.GeoDataFrame(crs=str(org_img.crs))
for geometrie in geometries_to_proces:
    file = gpd.read_file('geometries_out/'+geometrie)
    lista_toponimos_final = lista_toponimos_final.append(file)
lista_toponimos_final.reset_index(drop=True, inplace=True)
lista_toponimos_final.to_file('text_detected.geojson',driver='GeoJSON')